# Математический анализ в контексте задач оптимизации

### Метод Ньютона

In [2]:
# Описание функции
def f_0(x):
    return 3*x**2 - 6*x -45
# Описание производной функции
def f_1(x):
    return 6*x - 6

In [76]:
# Ручная реализация метода Ньютона
def newtons_method(f, der, eps, init):
    iter_count = 0
    x_curr = init
    f_1 = f(x_curr)
    while (abs(f_1) > eps):
        f_1 = f(x_curr)
        f_der = der(x_curr)
        x_curr = x_curr - (f_1)/(f_der)
        iter_count += 1
    return x_curr

In [77]:
init_value = 42
epsilon = 0.0001

newtons_method(f_0, f_1, epsilon, init_value)

5.000000000000001

In [3]:
# Стандартная реализация в библиотеке scipy
from scipy.optimize import newton
newton(func=f_0,x0=50,fprime=f_1, tol=0.0001)

5.0

### Квазиньютоновские методы

In [36]:
import numpy as np
from scipy.optimize import minimize

In [52]:
# Метод BFGS
# Описание функции
def func(x):
    return x[0]**2.0 -x[0]*x[1] + x[1]**2.0 + x[0]*9 - x[1]*6 + 20
# Описание градиента функции
def grad_func(x):
    return np.array([x[0]*2 - x[1] + 9, x[1]*2]-  x[0] - 6)

In [53]:
# Определим алгоритм:
x_0 = [-400, -400]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# Выведем результаты оптимизации
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 11
Решение: f([3.10850139e-07 3.00000004e+00]) = 11.00000


In [54]:
# то же самое с вариацией  L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# Выведем результаты оптимизации
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации ABNORMAL_TERMINATION_IN_LNSRCH
Количество оценок: 68
Решение: f([-1.22115751  3.29005994]) = 5.60262


# Задачи линейного программирования

### Библиотека Scipy

In [56]:
values = [4, 2, 1, 7, 3, 6] # стоимости товаров
weights = [5, 9, 8, 2, 6, 5] # вес товаров
C = 15 # вместимость сумки
n = 6 # количество товаров

In [58]:
c = - np.array(values) # изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  # конвертируем список с весами в массив
A = np.expand_dims(A, 0) # преобразуем размерность массива
b = np.array([C]) # конвертируем вместимость в массив

In [61]:
from scipy.optimize import linprog # для решения задач линейного программирования
linprog(c=c,
        A_ub=A, # ограничения, заданные двумерными массивами
        b_ub=b # ограничения, заданные одномерными массивами
)

     con: array([], dtype=float64)
     fun: -52.50000000003075
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24904539e-11])
  status: 0
 success: True
       x: array([6.18738537e-14, 1.05853307e-12, 1.21475944e-13, 7.50000000e+00,
       4.00246695e-13, 4.71394166e-13])

### Библиотека CVXPY (для задач целочисленного линейного программирования)

In [62]:
import cvxpy

In [71]:
# С помощью CVXPY создадим переменную-массив. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()
x.value

c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer linear programming
                    solver `GLPK`. If your problem is nonlinear then you can install SCIP
                    (pip install pyscipopt).

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

### Библиотека PulP

In [72]:
from pulp import *

In [73]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


In [74]:
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

In [75]:
c = cost.flatten()